# Water Mask

In [ ]:
import os
import numpy as np
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import random
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from segmentation_models_pytorch import Unet

In [ ]:

# Initialize the model
unet_model_water = Unet(
    encoder_name="efficientnet-b3",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=1  # Number of output classes
)


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = unet_model_water(input_tensor)
print(output.shape)

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.masks_dir, self.image_filenames[idx].replace('.jpg', '')+'_mask.png')  # Adjust file extension if needed

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Debugging prints
        if image is None:
            raise ValueError(f"Image not found or unable to load: {image_path}")
        if mask is None:
            raise ValueError(f"Mask not found or unable to load: {mask_path}")

        # Ensure proper dimensions
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError(f"Unexpected image shape: {image.shape}")

        # Resize or pad image and mask to be divisible by 32
        # image = self.resize_or_pad(image)
        # mask = self.resize_or_pad(mask)

        # Normalize image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # mask = mask // 51  # Adjust if necessary
        mask = np.where(mask == 4, 1, 0).astype('float32')
        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32).permute(0, 1, 2), torch.tensor(mask, dtype=torch.float32)



In [ ]:
test_images_dir = 'wildfire-week8/test/original'
test_masks_dir = 'wildfire-week8/test/mask'
# Create datasets
test_dataset_water = SegmentationDataset(test_images_dir, test_masks_dir, transform)

# Create data loaders
test_loader_water = DataLoader(test_dataset_water, batch_size=4, shuffle=False)

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

def save_predictions_with_overlays(model, model_state_path, test_loader, output_folder):
    """
    Displays images with overlayed ground truth and predicted masks, saving only the predicted overlay images.

    Parameters:
    - model: Trained PyTorch model used for predictions.
    - model_state_path: Path to the .pth file containing the model's state dictionary.
    - test_loader: DataLoader providing test images and masks.
    - output_folder: Folder where overlayed images will be saved.
    """
    # Load model weights from the .pth file
    model.load_state_dict(torch.load(model_state_path))
    model.eval()  # Set the model to evaluation mode
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    
    water_images = []
    water_masks = []
    
    # Collect images with water (mask has value 1)
    for images, masks in test_loader:
        for i in range(images.size(0)):  # Iterate over batch
            water_images.append(images[i].unsqueeze(0))
            water_masks.append(masks[i].unsqueeze(0))
    
    if len(water_images) == 0:
        print("No water images found in the test set.")
        return
    
    # Convert lists to tensors
    water_images = torch.cat(water_images, dim=0)
    water_masks = torch.cat(water_masks, dim=0)
    
    # Define the new mask color as RGB normalized values
    new_color = np.array([15/255, 94/255, 156/255])  # Normalize RGB values for 0f5e9c
    alpha = 0.9  # Transparency factor

    with torch.no_grad():
        for idx in range(len(water_images)):
            image = water_images[idx].to(torch.float32)  # Get the image
            mask = water_masks[idx].squeeze(0)  # Get the corresponding mask
            
            output = model(image.unsqueeze(0))  # Get prediction from the model
            prediction = (output > 0.5).float().squeeze(0)  # Binarize the output (threshold = 0.5)
            
            # Prepare image, mask, and prediction for overlay
            img_np = image.squeeze().permute(1, 2, 0).cpu().numpy()  # (C, H, W) -> (H, W, C)
            img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())  # Normalize to [0, 1]
            
            mask_np = mask.cpu().numpy()  # Convert mask to numpy
            pred_np = prediction.cpu().numpy().squeeze()  # Convert prediction to numpy and remove extra dimensions
            
            # Overlay mask onto original image (use custom color)
            img_with_mask = img_np.copy()
            img_with_mask[mask_np == 1] = new_color  # Apply custom color for ground truth mask
            img_with_mask = (alpha * img_with_mask + (1 - alpha) * img_np)  # Blend with original image
            
            img_with_pred = img_np.copy()
            img_with_pred[pred_np == 1] = new_color  # Apply the predicted mask with the custom color
            img_with_pred = (alpha * img_with_pred + (1 - alpha) * img_np)  # Blend with original image

            # Save only the predicted overlay image
            img_filename = f"predicted_overlay_{idx}.png"
            img_path = os.path.join(output_folder, img_filename)

            # Plot all three images
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))
            
            # Display original image
            axs[0].imshow(img_np)
            axs[0].set_title('Original Image')
            axs[0].axis('off')
            
            # Display original image with ground truth mask
            axs[1].imshow(img_with_mask)
            axs[1].set_title('Original Image with Ground Truth Mask')
            axs[1].axis('off')
            
            # Display original image with predicted mask
            axs[2].imshow(img_with_pred)
            axs[2].set_title('Original Image with Predicted Mask')
            axs[2].axis('off')
            
            plt.tight_layout()
            plt.show()  # Show the three images
            
            # Save the predicted overlay image only
            plt.imsave(img_path, img_with_pred)  # Save the predicted overlay image
            plt.close(fig)  # Close the figure to avoid display


In [ ]:
model_path = "best_unet_water_model_week8.pth"
output_folder = "wildfire-week8/test/water-overlay"
# Call the function to visualize random images
save_predictions_with_overlays(unet_model_water, model_path, test_loader_water, output_folder)

# Alive Tree Mask

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def color_based_segmentation(image):
    """Generates a mask for alive trees based on color segmentation."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 40, 20])   # Lowered brightness and saturation to capture darker greens
    upper_green = np.array([100, 255, 180])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    return green_mask

def process_images(original_folder, water_mask_folder, water_mask_applied_folder, output_format='png'):
    """Processes all images in the specified folders,
       applies tree detection methods, and visualizes results.
       
    Parameters:
    - output_format: The format to save the output images ('png', 'jpg', 'tiff', etc.)
    """
    
    # Get all original images
    original_images = [f for f in os.listdir(original_folder) if f.lower().endswith(('jpg', 'png', 'jpeg'))]
    
    # Loop through each original image file
    for image_file in original_images:
        # Read the original image
        original_image_path = os.path.join(original_folder, image_file)
        original_image = cv2.imread(original_image_path)
        if original_image is None:
            print(f"Error: Could not read original image from {original_image_path}.")
            continue

        # Read the corresponding water mask (e.g., predicted_mask_0.png)
        water_mask_file = f'predicted_mask_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_path = os.path.join(water_mask_folder, water_mask_file)
        water_mask = cv2.imread(water_mask_path, cv2.IMREAD_GRAYSCALE)

        if water_mask is None:
            print(f"Error: Could not read water mask from {water_mask_path}.")
            continue

        # Read the water mask applied image (e.g., predicted_overlay_1.png)
        water_mask_applied_file = f'predicted_overlay_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_applied_path = os.path.join(water_mask_applied_folder, water_mask_applied_file)
        water_mask_applied_image = cv2.imread(water_mask_applied_path)

        if water_mask_applied_image is None:
            print(f"Error: Could not read water mask applied image from {water_mask_applied_path}.")
            continue

        # Generate the mask for alive trees
        alive_tree_mask = color_based_segmentation(original_image)

        # Exclude areas identified as water in the water mask
        alive_tree_mask[water_mask > 0] = 0  # Set alive tree mask to 0 where water mask is present

        # Create transparent overlay for alive trees
        alive_tree_overlay = np.zeros_like(original_image, dtype=np.uint8)
        alive_tree_overlay[alive_tree_mask > 0] = [0, 255, 0]  # Green for alive trees

        # Combine overlays with the provided water mask applied image
        alpha_trees = 0.8  # Transparency for tree overlay
        final_overlay_image = cv2.addWeighted(water_mask_applied_image, 1, alive_tree_overlay, alpha_trees, 0)

        # Save the final overlay image
        output_filename = f'final_overlay_{os.path.splitext(image_file)[0]}.{output_format}'
        output_path = os.path.join(water_mask_applied_folder, output_filename)  # Save in the same folder
        cv2.imwrite(output_path, final_overlay_image)

        # Visualize the original image, water mask applied image, and final image with tree overlay
        plt.figure(figsize=(15, 5))

        # Show original image
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')

        # Show water mask applied image
        plt.subplot(1, 3, 2)
        plt.imshow(cv2.cvtColor(water_mask_applied_image, cv2.COLOR_BGR2RGB))
        plt.title('Image with Water Mask Applied')
        plt.axis('off')

        # Show final image with both overlays
        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(final_overlay_image, cv2.COLOR_BGR2RGB))
        plt.title('Final Image with Alive Tree Overlay')
        plt.axis('off')

        plt.show()

    print(f"Final overlay images saved in '{water_mask_applied_folder}' with format '{output_format}'.")



In [ ]:
# Example usage
original_folder = 'wildfire-week8/test/testloader-original'        # Change this to your image folder path
water_mask_folder = 'wildfire-week8/test/testloader-predict'
water_mask_applied_folder = 'wildfire-week8/test/water-overlay'# Change this to your water mask folder path
# output_folder = 'wildfire-week8/test/hsv'  
process_images(original_folder, water_mask_folder, water_mask_applied_folder)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def display_image_from_path(image_path):
    """
    Display an image from a given file path.

    Parameters:
    image_path (str): The file path to the image.
    """
    # Load the image from the path and convert to an RGB numpy array
    image = Image.open(image_path).convert("RGB")
    rgb_array = np.array(image)
    
    original_image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2HSV)
    hsv_array = np.array(hsv_image)
    # print(rgb_array[0])
    print(hsv_array[320, -100:])
    # # Display the image
    # plt.imshow(rgb_array)
    # plt.axis('off')  # Hide axes for a cleaner look
    # plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def process_image(image_path):
    """
    Processes a single image, applies an alive tree mask, and visualizes the original and masked images.

    Parameters:
    - image_path (str): Path to the original image.
    """
    
    # Read the original image
    original_image = cv2.imread(image_path)
    if original_image is None:
        print(f"Error: Could not read image from {image_path}.")
        return

    # Generate alive tree mask using a color-based segmentation function
    alive_tree_mask = color_based_segmentation(original_image)

    # Create an overlay for alive trees
    alive_tree_overlay = np.zeros_like(original_image, dtype=np.uint8)
    alive_tree_overlay[alive_tree_mask > 0] = [0, 255, 0]  # Green color for alive trees

    # Combine the original image with the tree mask overlay
    alpha_trees = 0.8
    final_image_with_overlay = cv2.addWeighted(original_image, 1, alive_tree_overlay, alpha_trees, 0)

    # Visualize the original and masked images
    plt.figure(figsize=(10, 5))

    # Show original image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    # Show image with alive tree overlay
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(final_image_with_overlay, cv2.COLOR_BGR2RGB))
    plt.title('Image with Alive Tree Overlay')
    plt.axis('off')

    plt.show()


In [ ]:
process_image('wildfire-week8/test/Layer 0.png')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def increase_highlights_auto_threshold(image_path, output_path, highlight_amount=2, percentile=50):
    """
    Increases highlights in the image by brightening the brightest regions.
    Automatically determines a threshold based on the brightness distribution.
    
    Parameters:
    - image_path: Path to the image file.
    - highlight_amount: Multiplier for highlight brightness (e.g., 1.2 = 20% increase).
    - percentile: Percentile to use for determining the brightness threshold.
    
    Returns:
    - Processed image with increased highlights.
    """
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

    # Convert to HSV (Hue, Saturation, Value) for better control over brightness
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)

    # Calculate the percentile threshold based on brightness values
    threshold = np.percentile(v, percentile)

    # Increase brightness only for pixels above the threshold
    highlight_mask = v > threshold
    v[highlight_mask] = np.clip(v[highlight_mask] * highlight_amount, 0, 255)

    # Merge channels back and convert to RGB
    hsv_highlighted = cv2.merge([h, s, v])
    highlighted_image = cv2.cvtColor(hsv_highlighted, cv2.COLOR_HSV2RGB)

    # Save the processed image
    highlighted_image_bgr = cv2.cvtColor(highlighted_image, cv2.COLOR_RGB2BGR)  # Convert back to BGR for saving
    cv2.imwrite(output_path, highlighted_image)

    # Display images
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    # plt.title("Original Image")
    # plt.axis('off')

    # plt.subplot(1, 2, 2)
    # plt.imshow(highlighted_image)
    # plt.title("Image with Increased Highlights")
    # plt.axis('off')
    # plt.show()

    return highlighted_image

# Example usage
increase_highlights_auto_threshold("wildfire-week8/test/testloader-original/original_image_21.png","wildfire-week8/test/highlighted.png")


In [ ]:
process_image('wildfire-week8/test/highlighted.png')

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def filter_and_visualize_high_contrast_images(folder_path, contrast_threshold):
    """
    Filters and visualizes images with high contrast from a specified folder.
    
    Parameters:
    - folder_path: Path to the folder containing images.
    - contrast_threshold: Threshold for standard deviation to consider an image as high contrast.
    
    Returns:
    - List of file paths of high-contrast images.
    """
    high_contrast_images = []
    
    # Loop through each image in the folder
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            continue  # Skip non-image files or corrupted images

        # Convert to grayscale for contrast calculation
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Calculate standard deviation of pixel intensities
        std_dev = np.std(gray_image)
        
        # Check if the image meets the high-contrast threshold
        if std_dev >= contrast_threshold:
            high_contrast_images.append(image_path)
    print(len(high_contrast_images))
    # Visualize the high-contrast images
    plt.figure(figsize=(15, 8))
    for i, img_path in enumerate(high_contrast_images):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

        # Display each high-contrast image
        plt.subplot(1, len(high_contrast_images), i + 1)
        plt.imshow(img_rgb)
        plt.title(f"High Contrast Image {i+1}")
        plt.axis('off')

    plt.show()

    return high_contrast_images


In [ ]:
# Example usage
folder_path = "wildfire-week8/test/testloader-original/"
high_contrast_images = filter_and_visualize_high_contrast_images(folder_path, contrast_threshold=70)

In [ ]:
# Example usage
increase_highlights_auto_threshold("wildfire-week8/test/testloader-original/original_image_9.png","wildfire-week8/test/highlighted9.png")

In [ ]:
# Example usage
increase_highlights_auto_threshold("wildfire-week8/test/testloader-original/original_image_53.png","wildfire-week8/test/highlighted53.png")

In [ ]:
process_image('wildfire-week8/test/highlighted9.png')

In [ ]:
process_image('wildfire-week8/test/highlighted53.png')

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def color_based_segmentation(image):
    """Generates a mask for alive trees based on color segmentation."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 40, 20])   # Lowered brightness and saturation to capture darker greens
    upper_green = np.array([100, 255, 180])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    return green_mask

def filter_and_visualize_high_contrast_images(folder_path, contrast_threshold):
    """
    Filters and visualizes images with high contrast from a specified folder.
    Parameters:
    - folder_path: Path to the folder containing images.
    - contrast_threshold: Threshold for standard deviation to consider an image as high contrast.
    Returns:
    - List of file paths of high-contrast images.
    """
    high_contrast_images = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        if image is None:
            continue
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        std_dev = np.std(gray_image)
        if std_dev >= contrast_threshold:
            high_contrast_images.append(image_path)
    
    # Visualize the high-contrast images
    plt.figure(figsize=(15, 8))
    for i, img_path in enumerate(high_contrast_images):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, len(high_contrast_images), i + 1)
        plt.imshow(img_rgb)
        plt.title(f"High Contrast Image {i+1}")
        plt.axis('off')
    plt.show()
    
    return high_contrast_images

# def increase_highlights_auto_threshold(image_path, output_path, highlight_amount=2, percentile=50):
#     """
#     Increases highlights in the image by brightening the brightest regions.
#     Automatically determines a threshold based on the brightness distribution.
#     Parameters:
#     - image_path: Path to the image file.
#     - highlight_amount: Multiplier for highlight brightness (e.g., 1.2 = 20% increase).
#     - percentile: Percentile to use for determining the brightness threshold.
#     """
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#     h, s, v = cv2.split(hsv)
#     threshold = np.percentile(v, percentile)
#     highlight_mask = v > threshold
#     v[highlight_mask] = np.clip(v[highlight_mask] * highlight_amount, 0, 255)
#     hsv_highlighted = cv2.merge([h, s, v])
#     highlighted_image = cv2.cvtColor(hsv_highlighted, cv2.COLOR_HSV2RGB)
#     highlighted_image_bgr = cv2.cvtColor(highlighted_image, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(output_path, highlighted_image_bgr)
#     return highlighted_image_bgr

def process_images(original_folder, water_mask_folder, water_mask_applied_folder, output_format='png', contrast_threshold=30):
    """
    Processes all images in the specified folders, applies tree detection methods, 
    and visualizes results after filtering high-contrast images and enhancing highlights.
    """
    # Step 1: Filter for high-contrast images
    high_contrast_images = filter_and_visualize_high_contrast_images(original_folder, contrast_threshold)

    # Step 2: Loop through each high-contrast image
    for image_path in high_contrast_images:
        image_file = os.path.basename(image_path)
        
        # Enhance highlights in the high-contrast image
        highlighted_image_path = os.path.join(original_folder, f"highlighted_{image_file}")
        highlighted_image = increase_highlights_auto_threshold(image_path, highlighted_image_path)
        print(highlighted_image_path)

        # Read the corresponding water mask (e.g., predicted_mask_0.png)
        water_mask_file = f'predicted_mask_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_path = os.path.join(water_mask_folder, water_mask_file)
        water_mask = cv2.imread(water_mask_path, cv2.IMREAD_GRAYSCALE)

        if water_mask is None:
            print(f"Error: Could not read water mask from {water_mask_path}.")
            continue

        # Read the water mask applied image (e.g., predicted_overlay_1.png)
        water_mask_applied_file = f'predicted_overlay_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_applied_path = os.path.join(water_mask_applied_folder, water_mask_applied_file)
        water_mask_applied_image = cv2.imread(water_mask_applied_path)

        if water_mask_applied_image is None:
            print(f"Error: Could not read water mask applied image from {water_mask_applied_path}.")
            continue

        # Use color-based segmentation to detect alive trees
        alive_tree_mask = color_based_segmentation(highlighted_image)

        # Exclude areas identified as water in the water mask
        alive_tree_mask[water_mask > 0] = 0  # Set alive tree mask to 0 where water mask is present

        # Create transparent overlay for alive trees
        alive_tree_overlay = np.zeros_like(highlighted_image, dtype=np.uint8)
        alive_tree_overlay[alive_tree_mask > 0] = [0, 255, 0]  # Green for alive trees

        # Combine overlays with the provided water mask applied image
        alpha_trees = 1
        final_overlay_image = cv2.addWeighted(water_mask_applied_image, 1, alive_tree_overlay, alpha_trees, 0)

        # Save the final overlay image
        output_filename = f'final_overlay_{os.path.splitext(image_file)[0]}.{output_format}'
        output_path = os.path.join(water_mask_applied_folder, output_filename)
        cv2.imwrite(output_path, final_overlay_image)

        # Visualize the original image, water mask applied image, and final image with tree overlay
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(highlighted_image)
        plt.title('Processed Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(final_overlay_image, cv2.COLOR_BGR2RGB))
        plt.title('Final Image with Alive Tree Overlay')
        plt.axis('off')
        
        plt.show()

    print(f"Final overlay images saved in '{water_mask_applied_folder}' with format '{output_format}'.")


In [ ]:
original_folder = 'wildfire-week8/test/testloader-original'        # Change this to your image folder path
water_mask_folder = 'wildfire-week8/test/testloader-predict'
water_mask_applied_folder = 'wildfire-week8/test/water-overlay'

process_images(original_folder, water_mask_folder, water_mask_applied_folder, output_format='png', contrast_threshold=70)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def process_images_trees(original_folder, contrast_threshold=30, output_format='png'):
    """
    Processes all images in the specified folder, applies tree detection methods,
    and visualizes results after filtering high-contrast images and enhancing highlights.
    """
    # Step 1: Filter for high-contrast images
    high_contrast_images = filter_and_visualize_high_contrast_images(original_folder, contrast_threshold)

    # Step 2: Loop through each high-contrast image
    for image_path in high_contrast_images:
        image_file = os.path.basename(image_path)
        
        # Enhance highlights in the high-contrast image
        highlighted_image_path = os.path.join(original_folder, f"highlighted_{image_file}")
        highlighted_image = increase_highlights_auto_threshold(image_path, highlighted_image_path)
        print(f"Highlighted image saved at: {highlighted_image_path}")

        # Use color-based segmentation to detect alive trees on the highlighted image
        alive_tree_mask = color_based_segmentation(highlighted_image)

        # Create transparent overlay for alive trees
        alive_tree_overlay = np.zeros_like(highlighted_image, dtype=np.uint8)
        alive_tree_overlay[alive_tree_mask > 0] = [0, 255, 0]  # Green for alive trees

        # Combine overlays with the highlighted image
        alpha_trees = 1.0
        final_overlay_image = cv2.addWeighted(highlighted_image, 1, alive_tree_overlay, alpha_trees, 0)

        # Save the final overlay image
        output_filename = f'final_overlay_{os.path.splitext(image_file)[0]}.{output_format}'
        output_path = os.path.join(original_folder, output_filename)
        cv2.imwrite(output_path, final_overlay_image)

        # Visualize the original image, highlighted image, and final overlay image
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(highlighted_image)
        plt.title('Highlighted Image')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(final_overlay_image)
        plt.title('Final Image with Alive Tree Overlay')
        plt.axis('off')
        
        plt.show()

    print(f"Final overlay images saved in '{original_folder}' with format '{output_format}'.")



In [ ]:
original_folder = 'wildfire-week8/valid/original' 
process_images_trees(original_folder, contrast_threshold=70)

In [ ]:
process_image('wildfire-week8/valid/original/DJI_0554_JPG.rf.6396d4a2947f673a85d2defb0b535bb6.jpg')